# Scraping whatever's in that barrel
This notebook scrapes the data from http://results.thecaucuses.org

Scraping is a little difficult because the data is not in a standard HTML table, but a bunch of nested divs.  Nonetheless, beautifulsoup made quick work of it.

The result is a table with two nested indices.  The columns are nested by candidate and the type of vote tally, and the rows are nested by county and precinct.

Hope you find this useful!

In [1]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from requests import get

In [2]:
# Grab HTML
url = 'https://results.thecaucuses.org/'
response = get(url)

In [3]:
# Parse it
html_soup = BeautifulSoup(response.text, 'html.parser')
type(html_soup)

bs4.BeautifulSoup

In [4]:
# Get the headers columns indicating the candidate and the type of vote tally
header = [h.text for h in html_soup.find('ul', class_ = 'thead')]
subhead = [h.text for h in html_soup.find('ul', class_ = 'sub-head')]

new_header = []
for h, s in zip(header, subhead):
    if h:
        ch = h
    new_header.append(ch)

header = new_header[2:]
subhead = subhead[2:]

col_index = pd.MultiIndex.from_arrays([header, subhead], names=('candidate', 'subhead'))
col_index

MultiIndex(levels=[['Bennet', 'Biden', 'Bloomberg', 'Buttigieg', 'Delaney', 'Gabbard', 'Klobuchar', 'Other', 'Patrick', 'Sanders', 'Steyer', 'Uncommitted', 'Warren', 'Yang'], ['Final Expression', 'First Expression', 'SDE']],
           codes=[[0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, 8, 8, 8, 9, 9, 9, 10, 10, 10, 12, 12, 12, 13, 13, 13, 7, 7, 7, 11, 11, 11], [1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2]],
           names=['candidate', 'subhead'])

In [5]:
# Pick out all of the divs that contain data for a single county
counties = html_soup.find_all('div', class_ = 'precinct-rows')

In [6]:
# Pull data for each precinct out, assembling the multiindex as it goes
ind_county = []
ind_precinct = []
votes = []

for row in counties:
    county_name, data = row.children
    county_name = county_name.text
    
    for row in data.children:
        dat = [c.text for c in row]
        
        ind_county.append(county_name)
        ind_precinct.append(dat[0])
        
        votes.append([float(d.replace(',', '')) for d in dat[1:]])

row_index = pd.MultiIndex.from_arrays([ind_county, ind_precinct], names=('county', 'precinct'))


In [7]:
# Convert the data into a dataframe
results = pd.DataFrame(votes, columns=col_index, index=row_index)
results

candidate                                                                  Bennet  \
subhead                                                          First Expression   
county        precinct                                                              
Adair         4SE ORIENT                                                      0.0   
              1NW ADAIR                                                       0.0   
              2NE STUART                                                      0.0   
              5GF GREENFIELD                                                  0.0   
              3SW FONTANELLE                                                  0.0   
              Total                                                           0.0   
Adams         Adams 5                                                         0.0   
              Adams 2                                                         0.0   
              Adams 1                                                         0.0   
              Adams 3                                                         0.0   
              Adams 4                                                         0.0   
              Total                                                           0.0   
Allamakee     Pct 07 - CN/LF/LS/LS City                                       0.0   
              Pct 05 - LT/PC/WV City                                          0.0   
              Pct 03 - PO/FK                                                  0.0   
              Pct 04 - PV City                                                0.0   
              Pct 08 - IA/UC/NA City                                          0.0   
              Pct 10 - Waukon 2                                               0.0   
              Pct 02 - FC/JF/LL/MK/UP                                         0.0   
              Pct 01 - WL/HV                                                  0.0   
              Pct 11 - Waukon 3                                               1.0   
              Pct 09 - Waukon 1                                               0.0   
              Total                                                           1.0   
Appanoose     Walnut                                                          0.0   
              Bellair/Lincoln                                                 0.0   
              Caldwell Township                                               0.0   
              Pleasant/Franklin                                               0.0   
              Centerville Ward 1                                              0.0   
              Taylor/Chariton                                                 0.0   
              Union / Udell Township                                          0.0   
...                                                                           ...   
CD2 Satellite Davenport, IA - Humility of Mary Center                         0.0   
              Iowa City, IA - Oaknoll Retirement Services                     0.0   
              Keokuk, IA - Keokuk Labor Temple                                0.0   
              Davenport, IA - Scott Community College                         0.0   
              Ottumwa, IA - UAW Local 74                                      0.0   
              Ft. Madison, IA - LULAC Club                                    0.0   
              Iowa City, IA - Phillips Hall at University of ...              0.0   
              Iowa City, IA - Iowa City Fire Station 2                        0.0   
              West Burlington, IA - Southeastern Community Co...              0.0   
              Iowa City, IA - University of Iowa Hospitals & ...              0.0   
              Centerville, IA - Centerville Speciality Care                   0.0   
              Ottumwa, IA - UFCW Local 230                                    0.0   
              Albia, IA - Monroe Care Center                                  0.0   
     

In [8]:
# save it!
results.to_csv('iowadems_resumts_0205_8PM.csv')

# And with a flattened row index (in case the one above is not readable)
results.reset_index().to_csv('iowadems_resumts_0205_8PM_rowunwrapped.csv')